In [67]:
import pandas as pd
import requests as r
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import plotly.express as px
from utils.settings import DETERMINISTIC_MODELS, DETERMINISTIC_VARS

In [68]:
def get_historical_data(latitude=53.55,
                        longitude=9.99,
                        variables='temperature_2m',
                        timezone='auto',
                        model='best_match',
                        start_date='1991-01-01',
                        end_date='2020-12-31'):
    """
    Get historical data for a point
    """
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": variables,
        "timezone": timezone,
        "models": model,
        "start_date": start_date,
        "end_date": end_date
    }

    resp = r.get("https://archive-api.open-meteo.com/v1/archive",
                 params=payload)
    data = pd.DataFrame.from_dict(resp.json()['hourly'])
    data['time'] = pd.to_datetime(
        data['time'], format='%Y-%m-%dT%H:%M')

    data = data.dropna()

    return data

In [75]:
def compute_monthly_clima(latitude=53.55, longitude=9.99, model='era5',
                          start_date='1991-01-01', end_date='2020-12-31'):
    """Takes a 30 years hourly dataframe as input and compute
    some monthly statistics by aggregating many times"""

    df = get_historical_data(
        variables='temperature_2m,precipitation,snowfall,cloudcover,windspeed_10m',
        latitude=latitude,
        longitude=longitude,
        timezone='GMT',  # Use UTC to avoid issues with daily computations
        model=model,
        start_date=start_date,
        end_date=end_date)

    daily = df.resample('1D', on='time').agg(t2m_mean=('temperature_2m', 'mean'),
                                             t2m_min=('temperature_2m', 'min'),
                                             t2m_max=('temperature_2m', 'max'),
                                             wind_speed_max=(
                                                 'windspeed_10m', 'max'),  # in km/h
                                             daily_rain=(
                                                 'precipitation', 'sum'),  # in mm
                                             daily_snow=(
                                                 'snowfall', 'sum'),  # in cm
                                             cloudcover_mean=(
                                                 'cloudcover', 'mean')  # in %
                                             )
    daily['overcast'] = daily['cloudcover_mean'] >= 80
    daily['partly_cloudy'] = (daily['cloudcover_mean'] < 80) & (
        daily['cloudcover_mean'] > 20)
    daily['sunny'] = daily['cloudcover_mean'] <= 20
    daily['t_max_gt_30'] = daily['t2m_max'] > 30
    daily['t_max_gt_25'] = (daily['t2m_max'] > 25) & (daily['t2m_max'] <= 30)
    daily['t_max_gt_20'] = (daily['t2m_max'] > 20) & (daily['t2m_max'] <= 25)
    daily['t_max_gt_15'] = (daily['t2m_max'] > 15) & (daily['t2m_max'] <= 20)
    daily['t_max_gt_10'] = (daily['t2m_max'] > 10) & (daily['t2m_max'] <= 15)
    daily['t_max_gt_5'] = (daily['t2m_max'] > 5) & (daily['t2m_max'] <= 10)
    daily['t_max_gt_0'] = (daily['t2m_max'] >= 0) & (daily['t2m_max'] <= 5)
    daily['t_max_lt_0'] = (daily['t2m_max'] >= -5) & (daily['t2m_max'] < 0)
    daily['t_max_lt_m5'] = daily['t2m_max'] < -5
    daily['frost'] = daily['t2m_min'] <= 0
    daily['wet'] = daily['daily_rain'] >= 1.0  # mm
    daily['dry'] = daily['daily_rain'] < 1.0  # mm
    daily['snow'] = daily['daily_snow'] >= 1.0  # cm
    daily['p_50_100'] = (daily['daily_rain'] <= 100) & (
        daily['daily_rain'] > 50)
    daily['p_20_50'] = (daily['daily_rain'] <= 50) & (daily['daily_rain'] > 20)
    daily['p_10_20'] = (daily['daily_rain'] <= 20) & (daily['daily_rain'] > 10)
    daily['p_5_10'] = (daily['daily_rain'] <= 10) & (daily['daily_rain'] > 5)
    daily['p_2_5'] = (daily['daily_rain'] <= 5) & (daily['daily_rain'] > 2)
    daily['p_lt_2'] = (daily['daily_rain'] <= 2) & (daily['daily_rain'] > 1)
    daily['w_gt_61'] = (daily['wind_speed_max'] >= 61)
    daily['w_gt_50'] = (daily['wind_speed_max'] >= 50) & (daily['wind_speed_max'] < 61)
    daily['w_gt_38'] = (daily['wind_speed_max'] >= 38) & (daily['wind_speed_max'] < 50)
    daily['w_gt_28'] = (daily['wind_speed_max'] >= 28) & (daily['wind_speed_max'] < 38)
    daily['w_gt_19'] = (daily['wind_speed_max'] >= 19) & (daily['wind_speed_max'] < 28)
    daily['w_gt_12'] = (daily['wind_speed_max'] >= 12) & (daily['wind_speed_max'] < 19)
    daily['w_gt_5'] = (daily['wind_speed_max'] >= 5) & (daily['wind_speed_max'] < 12)
    daily['w_gt_1'] = (daily['wind_speed_max'] >= 1) & (daily['wind_speed_max'] < 5)
    daily['w_calm'] = daily['wind_speed_max'] <= 0.1
    #
    bool_cols = daily.dtypes[daily.dtypes == bool].index
    # Compute monthly stats
    monthly = daily[bool_cols].resample('1M').sum().add_suffix('_days')
    monthly['monthly_rain'] = daily['daily_rain'].resample('1M').sum()
    monthly['t2m_max_mean'] = daily['t2m_max'].resample('1M').mean()
    monthly['t2m_min_mean'] = daily['t2m_min'].resample('1M').mean()
    monthly['t2m_min_min'] = daily['t2m_min'].resample('1M').min()
    monthly['t2m_max_max'] = daily['t2m_max'].resample('1M').max()
    stats = monthly.groupby(monthly.index.month).mean().round(1)

    return stats

In [78]:
stats = compute_monthly_clima()

In [96]:
end_date = (pd.to_datetime('now',utc=True)-pd.to_timedelta('2 days')).strftime("%Y-%m-%d")
start_date = pd.to_datetime('now',utc=True).replace(month=1, day=1).strftime("%Y-%m-%d")
stats_year = compute_monthly_clima(start_date=start_date, end_date=end_date)

In [110]:
x = ['Jan','Feb','Mar','Apr','May',
     'Jun','Jul','Aug','Sep','Oct',
     'Nov','Dec']

def make_winds_climate_figure(df, current=None):
     fig = make_subplots()
     traces = []

     traces.append(
     go.Bar(
          x=x,
          y=df['w_gt_61_days'],
          name='>61 km/h',
          text=df['w_gt_61_days'],
          textfont=dict(color='white'),
          opacity=0.9,
          marker=dict(color='rgb(245,134,38)'),
          showlegend=True,),
     )
     print(x[0:len(current)],)
     print(current['w_gt_38_days'])
     go.Scatter(
          x=x[0:len(current)],
          y=current['w_gt_38_days'],
          marker=dict(color='black', size=12),
          showlegend=True,
     )
     traces.append(
     go.Bar(
          x=x,
          y=df['w_gt_50_days'],
          name='>50 km/h',
          text=df['w_gt_50_days'],
          textfont=dict(color='white'),
          opacity=0.9,
          marker=dict(color='rgb(226,225,53)'),
          showlegend=True,),
     )
     traces.append(
     go.Bar(
          x=x,
          y=df['w_gt_38_days'],
          name='>38 km/h',
          text=df['w_gt_38_days'],
          textfont=dict(color='white'),
          opacity=0.9,
          marker=dict(color='rgb(122,171,36)'),
          showlegend=True,),
     )
     traces.append(
     go.Bar(
          x=x,
          y=df['w_gt_28_days'],
          name='>28 km/h',
          text=df['w_gt_28_days'],
          textfont=dict(color='white'),
          opacity=0.9,
          marker=dict(color='rgb(0,139,85)'),
          showlegend=True,),
     )
     traces.append(
     go.Bar(
          x=x,
          y=df['w_gt_19_days'],
          name='>19 km/h',
          text=df['w_gt_19_days'],
          textfont=dict(color='white'),
          opacity=0.9,
          marker=dict(color='rgb(0,184,44)'),
          showlegend=True,),
     )
     traces.append(
     go.Bar(
          x=x,
          y=df['w_gt_12_days'],
          name='>12 km/h',
          text=df['w_gt_12_days'],
          textfont=dict(color='white'),
          opacity=0.9,
          marker=dict(color='rgb(74,218,102)'),
          showlegend=True,),
     )
     traces.append(
     go.Bar(
          x=x,
          y=df['w_gt_5_days'],
          name='>5 km/h',
          text=df['w_gt_5_days'],
          textfont=dict(color='white'),
          opacity=0.9,
          marker=dict(color='rgb(113,247,153)'),
          showlegend=True,),
     )
     traces.append(
     go.Bar(
          x=x,
          y=df['w_gt_1_days'],
          name='>1 km/h',
          text=df['w_gt_1_days'],
          textfont=dict(color='white'),
          opacity=0.9,
          marker=dict(color='rgb(152,250,206)'),
          showlegend=True,),
     )
     traces.append(
     go.Bar(
          x=x,
          y=df['w_calm_days'],
          name='Calm',
          text=df['w_calm_days'],
          textfont=dict(color='white'),
          opacity=0.9,
          marker=dict(color='rgb(217,209,209)'),
          showlegend=True,),
     )

     for trace in traces:
          fig.add_trace(trace)

     fig.update_layout(
     margin={"r": 0.1, "t": 0.1, "l": 0.1, "b": 0.1},
     template='plotly_white',
     barmode='stack',
     legend=dict(orientation='h')
     )

     fig.update_yaxes(showgrid=False)
     fig.update_yaxes(showgrid=False)
     
     return fig

make_winds_climate_figure(stats, stats_year)

['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']
time
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    1.0
8    0.0
9    0.0
Name: w_gt_38_days, dtype: float64


In [7]:
def get_forecast_data(latitude=53.55,
                      longitude=9.99,
                      variables=",".join(DETERMINISTIC_VARS),
                      timezone='auto',
                      model=DETERMINISTIC_MODELS[0],
                      forecast_days = 7,
                      from_now=True):
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": variables,
        "timezone": timezone,
        "models": model,
        "forecast_days": forecast_days
    }

    resp = r.get("https://api.open-meteo.com/v1/forecast",
                 params=payload)
    data = pd.DataFrame.from_dict(resp.json()['daily'])
    data['time'] = pd.to_datetime(
        data['time']).dt.tz_localize(resp.json()['timezone'])

    # Optionally subset data to start only from previous hour
    if from_now:
        data = data[data.time >= pd.to_datetime(
            'now', utc=True).tz_convert(resp.json()['timezone']).floor('H')]

    return data

In [14]:
df = get_forecast_data(model="ecmwf_ifs04,gfs_seamless,jma_seamless,icon_seamless,gem_seamless,meteofrance_seamless",
                       longitude=22.9425,
                       latitude=39.361,
                       variables='precipitation_sum,precipitation_hours',
                       forecast_days=7,
                       from_now=False)

In [20]:
df = df.set_index('time')

In [45]:
fig = df.loc[:,df.columns.str.contains('precipitation_sum')].cumsum().plot(backend='plotly')
# fig.add_trace(
#     go.Scatter(
#         x=df.index,
#         y=df.loc[:,df.columns.str.contains('precipitation_hours')].values,
#         # yaxis="y2"
#     ))
fig. update_layout(showlegend=False)

In [48]:
def make_lineplot_timeseries(df, var, mode='lines+markers'):
    traces = []
    for col in df.columns[df.columns.str.contains(var)]:
        # First do the first 48 hrs
        traces.append(
            go.Scatter(
                x=df.loc[:, 'time'],
                y=df.loc[:, col],
                mode=mode,
                name=col,
                marker=dict(size=5),
                line=dict(width=2),
                showlegend=False),
        )

    return traces


def make_barplot_timeseries(df, var):
    traces = []
    for col in df.columns[df.columns.str.contains(var)]:
        traces.append(
            go.Bar(
                x=df['time'],
                y=df[col],
                name=col,
                opacity=0.6,
                marker=dict(color='rgb(26, 118, 255)'),
                showlegend=False),
        )

    return traces

def make_subplot_figure(data):
    traces_temp = make_lineplot_timeseries(data, 'temperature_2m')
    traces_precipitation = make_barplot_timeseries(data, 'precipitation')
    traces_wind = make_lineplot_timeseries(data, 'windspeed_10m')
    traces_cloud = make_lineplot_timeseries(data, 'cloudcover', mode='markers')

    fig = make_subplots(
        rows=4,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.5, 0.4, 0.3, 0.3])

    for trace_temp in traces_temp:
        fig.add_trace(trace_temp, row=1, col=1)
    for trace_precipitation in traces_precipitation:
        fig.add_trace(trace_precipitation, row=2, col=1)
    for trace_wind in traces_wind:
        fig.add_trace(trace_wind, row=3, col=1)
    for trace_cloud in traces_cloud:
        fig.add_trace(trace_cloud, row=4, col=1)

    fig.update_layout(
        xaxis=dict(showgrid=True,
                   range=[data['time'].min(),
                          data['time'].max()]),
        yaxis=dict(showgrid=True,),
        height=800,
        margin={"r": 0.1, "t": 0.1, "l": 0.1, "b": 0.1},
        template='plotly_white',
        barmode='overlay'
    )

    fig.update_yaxes(title_text="2m Temp [°C]", row=1, col=1)
    fig.update_yaxes(title_text="Prec. [mm]", row=2, col=1)
    fig.update_yaxes(title_text="Wind. [kph]", row=3, col=1)
    fig.update_yaxes(title_text="Cloud cover [%]", row=4, col=1)
    fig.update_yaxes(showgrid=True, gridwidth=4)
    fig.update_xaxes(minor=dict(ticks="inside", showgrid=True,
                     gridwidth=3), showgrid=True, gridwidth=4)

    return fig


make_subplot_figure(df)